In [1]:
## Ensure the correct working directory

import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime
from IPython.display import clear_output

In [3]:
from workflow.pipeline import culture, ephys, probe

[2025-06-13 10:27:33,974][INFO]: Connecting judewerth@db.datajoint.com:3306
[2025-06-13 10:27:34,684][INFO]: Connected judewerth@db.datajoint.com:3306


Determine which entries you'd like to delete

- To delete many entries: Change s to filter entires down to the one you'd like to delete, then fetch keys.

- To delete specific entries: Paste specific keys to be delelted

In [ ]:
# Remove Many Entries
# Use strings to filter down Ephys Sessions
no_data = "BETWEEN '2023-07-13 1:59:00' AND '2023-07-13 11:54:00'"
s = f"start_time {no_data} OR end_time {no_data}"

# Make sure you'd like to delete all sessions below
delete_sessions = ephys.EphysSession & s
delete_sessions

In [ ]:
# Fetch keys for those sessions
keys = delete_sessions.fetch("KEY")
keys

In [ ]:
# Delte specific keys
# Paste specific keys (dicts) in the list
keys = []

# Make sure you'd like to delete all sessions below
ephys.EphysSession & keys

Delete Ephys Sessions
- You will be prompted to delete Ephys Sessions
- There will likely be child entries to delete first (in other tables)
- To delete, type "yes" in the prompt at the top then press enter
- If you'd like to cancel, press "Esc"

In [ ]:
for key in keys:
    print(f"Deleting session: \n    {key}")

    info_query = ephys.EphysSessionInfo & key
    if len(info_query) > 0:
        print(f"{len(info_query)} Info Sessions Found")

        info_keys = info_query.fetch("KEY")
        (ephys.EphysSessionInfo & info_keys).delete()

    probe_query = (ephys.EphysSessionProbe & key)
    if len(probe_query) > 0:
        print(f"{len(probe_query)} EphysProbe Sessions Found")

        probe_keys = probe_query.fetch("KEY")
        (ephys.EphysSessionProbe & probe_keys).delete() 

    cluster_query = ephys.ClusteringTask & key
    if len(cluster_query) > 0:
        print(f"{len(cluster_query)} Clustering Tasks Found")

        cluster_keys = cluster_query.fetch("KEY")
        (ephys.ClusteringTask & cluster_keys).delete()

    lfp_query = ephys.LFP & key
    if len(lfp_query) > 0:
        print(f"{len(lfp_query)} LFP Sessions Found")

        lfp_keys = lfp_query.fetch("KEY")
        (ephys.LFP & lfp_keys).delete()

    print("Child tables deleted: Now delete the EphysSession")
    (ephys.EphysSession & key).delete()

    clear_output()

In [5]:
# if you get an error it is likely due to child tables having the key
# find them with
ephys.EphysSession.children()

['`utah_organoids_ephys`.`_ephys_session_info`',
 '`utah_organoids_ephys`.`_l_f_p`',
 '`utah_organoids_ephys`.`clustering_task`',
 '`utah_organoids_ephys`.`ephys_session_probe`']